In [ ]:
#https://github.com/hpcaitech/ColossalAI/tree/main/applications/Chat
git clone https://github.com/hpcaitech/ColossalAI.git
cd ColossalAI/applications/Chat
pip install .
cd transformers
pip install .
#https://anaconda.org/conda-forge/cudatoolkit-dev
conda install -c conda-forge cudatoolkit-dev
#https://github.com/NVIDIA/apex
conda install -c conda-forge nvidia-apex

In [ ]:
torchrun --standalone --nproc_per_node=1 train_sft.py \
    --pretrain "./checkpoint/llama_7B" \
    --model 'llama' \
    --strategy colossalai_zero2 \
    --log_interval 10 \
    --save_path "./checkpoint_mine/step1/epoch1" \
    --dataset 'PersonaChat' \
    --batch_size 2 \
    --accumulation_steps 8 \
    --lr 2e-5 \
    --max_epochs 1 \
    --max_datasets_size 1024 \
    --lora_rank 16 \
    --batch_size 2 \
    --max_datasets_size 16

In [ ]:
torchrun --standalone --nproc_per_node=1 train_sft.py \
    --pretrain 'facebook/opt-2.7b' \
    --model 'opt' \
    --strategy colossalai_zero2 \
    --log_interval 10 \
    --save_path './checkpoint_opt/step1/epoch1' \
    --dataset 'PersonaChat' \
    --batch_size 2 \
    --accumulation_steps 8 \
    --lr 2e-5 \
    --max_epochs 1 \
    --lora_rank 16 \
    --batch_size 4 \
    --max_datasets_size 16

In [ ]:
torchrun --standalone --nproc_per_node=1 train_reward_model.py \
    --strategy colossalai_zero2 \
    --model 'bloom' \
    --pretrain 'bigscience/bloom-560m' \
    --dataset 'PersonaChat' \
    --save_path './checkpoint_mine/step2/epoch1/rmstatic.pt' \
    --max_epochs 1 \
    --batch_size 8 \
    --loss_fn 'log_exp' \
    --max_datasets_size 16

In [ ]:
torchrun --standalone --nproc_per_node=1 train_reward_model.py \
    --strategy colossalai_zero2 \
    --model 'opt' \
    --pretrain './checkpoint_opt/step1/epoch1' \
    --dataset 'PersonaChat' \
    --save_path './checkpoint_opt/step2/epoch1/rmstatic.pt' \
    --max_epochs 1 \
    --batch_size 4 \
    --loss_fn 'log_exp' \
    --lora_rank 16 \
    --max_datasets_size 16

In [ ]:
torchrun --standalone --nproc_per_node=1 train_prompts.py \
    --prompt_dataset 'PersonaChat' \
    --pretrain_dataset 'PersonaChat' \
    --strategy colossalai_zero2 \
    --model 'llama' \
    --pretrain './checkpoint_mine/step1/epoch1' \
    --rm_model 'bloom' \
    --rm_pretrain 'bigscience/bloom-560m' \
    --rm_path './checkpoint_mine/step2/epoch1/rmstatic.pt' \
    --save_path './checkpoint_mine/step3/epoch1' \
    --num_episodes 1 \
    --max_epochs 1 \
    --train_batch_size 1 \
    --ptx_batch_size 1 \
    --lora_rank 16 \
    --max_input_len 511 \
    --max_seq_len 512 \
    --max_datasets_size 16

In [ ]:
torchrun --standalone --nproc_per_node=1 train_prompts.py \
    --prompt_dataset 'PersonaChat' \
    --pretrain_dataset 'PersonaChat' \
    --strategy colossalai_zero2 \
    --model 'llama' \
    --pretrain './checkpoint_mine/step1/epoch1' \
    --rm_model 'llama' \
    --rm_pretrain './checkpoint_mine/step1/epoch1' \
    --rm_path './checkpoint_mine/step2/epoch1/rmstatic.pt' \
    --save_path './checkpoint_mine/step3/epoch1/acstatic.pt' \
    --num_episodes 1 \
    --max_epochs 1 \
    --train_batch_size 1 \
    --ptx_batch_size 1 \
    --lora_rank 16 \
    --max_input_len 511 \
    --max_seq_len 512 \
    --max_datasets_size 16